# Dataset Overview for Bart tokenizer



In [1]:
#Install essential packages
%%capture
!pip install datasets
!pip install transformers

!apt install git-lfs

In [2]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers

In [3]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

##XSum

In [4]:
# import dataset from cache
train_xsum = datasets.load_dataset("xsum", split="train")

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


In [5]:
train_xsum

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})

In [6]:
# Visualize the Data

import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    del df["id"]
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(train_xsum)

,document,summary
0,"Robert Faulds was severely injured when his right arm was pulled into machinery at the bottle-top factory in Bridge of Allan in September 2013.\nThe 58-year-old said United Closures and Plastics blamed him for the accident and demoted him.\nThe firm was also fined £12,000 after admitting health and safety breaches at Stirling Sheriff Court.\nMr Faulds, from Falkirk, was working on a printing machine when the accident happened.\nHis arm was dragged into the offset machine and crushed.\nThe Health and Safety Executive took United Closures and Plastics to court over the incident, where the firm admitted failing to do a sufficient risk assessment of the machine and failing to ensure proper access.\nThe firm was issued the £12,000 fine at Stirling Sheriff Court last week.\nMr Faulds, who was off work for more than 14 months after the accident, said he was sacked from his role as production engineer for gross misconduct but was offered another job as an operator.\nThe father-of-four said: ""The thing that upsets me is that all the way through this, they have put the blame on me. They said I did an unsafe act and had a flippant attitude to safety.\n""My arm was smashed to pieces and I am now 30% disabled in it.\n""The money will change my life because I will be able to buy a flat and look to the future.\n""But I am now doing a menial job packing boxes when I was working as a skilled engineer before. But who is going to employ me when it's on my record that I was sacked for gross misconduct?\n""I have never had an apology but I would just like them to clear my record.""\nMr Faulds' lawyer, Tracey McKenzie from Thompsons Solicitors, said: ""Robert is a very hard-working man who was following instructions from his employer when he was seriously injured.\n""He was used as a scapegoat and sacked before being reinstated in a lower role.\n""His employers were then prosecuted and pled guilty to breaches of health and safety law which totally vindicates Robert's version of events.\n""He has now been left with a life-changing injury and his future employment prospects have been greatly affected - it is only right that he has been paid this substantial level of compensation.""\nNo-one from United Closures and Plastics was available for comment.","A factory worker who had his arm crushed by machinery has won a £125,000 payout from his employer."
1,"""Significant damage"" has been caused to the Churchfields playing fields in West Bromwich, according to Sandwell council.\nThe authority said ""a number of incidents"" have been reported since the travellers arrived on Wednesday.\n""They have ruined the football pitches and damaged metal palisade fencing,"" said Councillor Paul Moore.\nIn a statement the council said officers were ""working to move the travellers on as quickly as possible"".\nMr Moore, cabinet member for regeneration and investment, said: ""It's completely unacceptable.\n""It's also unfair on kids and their parents that these matches have been cancelled. I am sorry for the inconvenience that's been caused.\n""Illegal encampments will not be tolerated on council owned land and we will be working to get the pitches back to a decent standard very soon.""",Children's football matches have been cancelled after travellers set up an illegal camp on a playing field.
2,"The 35-year-old collapsed during a training session with his Gabonese club FC 105 Libreville.\nHe played for the Panthers at the Africa Cup of Nations in 2010 and 2012.\n""We played together, we played against each other but it was always a pleasure to play alongside you,"" Gabon captain Pierre-Emerick Aubameyang posted.\n""Rest in peace my brother.""\nThe Gabon Football Federation (Fegafoot) announced the news and also paid their own tribute.\n""The country has lost a brave son who has always given his best for the Gabonese flag,"" Fegafoot wrote.\n""All our condolences to his family and team-mates.""\nSunderland and Gabon midfielder Didier Ndong also pai

In [8]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["document"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["summary"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [9]:
# random choose 10% of the dataset
sample_size = 20000
data_stats = train_xsum.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (1140 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1334 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1126 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1078 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 489.4413, %-Articles > 512:0.34995, Summary Mean:28.1944, %-Summary > 64:0.002, %-Summary > 128:0.0


From the Result, we can see that for random 20000 sample (10% of training dataset), our articles has average token length of 489, and 35% is longer than 512. And the mean summary's token length is 28, almost none are longer than 64. 

It should be reasonable to use 512 for tokenized article length and 64 for tokenized summary length on Bart tokenizer for Xsum dataset. But since Bart is limited to 1024, and to keep the same track with other dataset, we will stick to **1024** and **128**.


##CNN/Daily

In [11]:
train_cnn = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


In [12]:
train_cnn

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [13]:
show_random_elements(train_cnn)

,article,highlights
0,"Former Arsenal title-winner Alan Smith believes the club's Premier League challenge this season is already over and the best that they can achieve is another top four finish. The Gunners left it late to defeat Southampton 1-0 at home on Wednesday night courtesy of an Alexis Sanchez strike - a result that moves the Gunners up to sixth in the Premier League. However, Smith feels the gap between the Emirates outfit and league leaders Chelsea is already insurmountable after 14 matches, despite Arsene Wenger's belief that they can still overhaul Jose Mourinho's side. Former Arsenal striker Alan Smith believes the club must forget about the title and focus on a top-four finish . Alexis Sanchez (right) scored in the 89th minute to give the Gunners a 1-0 win against Southampton . The Blues are unbeaten this season and sit top of the table on 36 points - 13 ahead of their bitter London rivals - and the former England striker is adamant Arsenal must set their sights lower and focus on achieving qualification for the Champions League next season. Speaking on TalkSPORT's Hawksbee and Jacobs show, the 1988-89 and 1990-91 First Division winner with Arsenal said: 'I can see them having a good run, they have done it in years gone by, but I can't see them getting the kind of results that give them that proper belief that they are title challengers. 'I think they are a better team going forward now, without question, but you look back to their defensive weaknesses and think of the Anderlecht game, the Swansea game, the Man United game and have they erased it from their game? 'We will have to see when they come up against better opposition in the months to come. I don't see them as a title-challenging team. 'They will finish in the top four, no danger. I think them and Manchester United will be vying for those third and fourth places, but I cannot see them getting to the level of Chelsea.' Loic Remy (left) was on target as Chelsea beat Tottenham 3-0 on Wednesday in a London derby .","Arsenal scored a late winner to beat Southampton 1-0 on Wednesday .\nResult moves Gunners up to sixth in the Premier League, 13 points adrift of leaders Chelsea after 14 games .\nArsenal travel to Stoke on Saturday in their next league encounter ."
1,"(CNN) -- Egyptian security forces have arrested four Al Jazeera journalists, with the government saying at least one of them met with members of the banned Muslim Brotherhood. Correspondent Peter Greste, Bureau Chief Mohamed Fahmy, producer Baher Mohamed and cameraman Mohamed Fawzy were taken into custody Sunday evening in Cairo, the network said. The Egyptian Interior Ministry said on its Facebook page that security forces arrested a Muslim Brotherhood member and an Australian journalist at a Cairo hotel. Greste, an Australian, previously worked for CNN, Reuters and the BBC. The ministry said the Muslim Brotherhood member used the hotel to meet with other members and as a media center to broadcast damaging news about the government for Al Jazeera. The network demanded the journalists be released. Calling the arrests arbitrary, Al Jazeera said the network ""has been subject to harassment by Egyptian security forces,"" including having its equipment confiscated and offices raided, despite the network not being banned from working in Egypt. The arrests occurred the day before the Committee to Protect Journalists issued a report saying Egypt, Syria and Iraq have become the deadliest countries for journalists. Seventy journalists were killed in 2013, with six of those deaths in Egypt, the CPJ said. Egypt's current round of political turmoil began when former President Mohamed Morsy was removed from office in a July 3 coup. Last week, the military-backed government declared the Muslim Brotherhood a terrorist organization. Police and members of the Muslim Brotherhood have clashed in the streets since then.","NEW: Four Al Jazeera journalists were arrested, the network said .\nNEW: Al Jazeera calls the a

In [14]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["article"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["highlights"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [15]:
sample_size = 25000
data_stats = train_cnn.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (1287 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1881 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2023 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1398 > 1024). Running this sequence through the model will result in indexing errors


In [16]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 822.216, %-Articles > 512:0.72224, Summary Mean:60.1818, %-Summary > 64:0.3936, %-Summary > 128:0.0


We can see that on average an article contains 822 tokens with 3/4 of the articles being longer than 512. The summary is on average 60 tokens long. 40% our 25000-sample summaries are longer than 64 tokens, and almost none are longer than 128 tokens.

Bart has limit of `max_length` 1024 on tokenizer, It should be reasonable to use **1024** for tokenized article length and **128** for tokenized summary length for CNN/Daily dataset.

##Wikihow

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
!ls /content/drive/MyDrive/dataset

wikihowAll.csv


In [17]:
# import dataset from cache
dataset_wiki = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset")

Using custom data configuration all-6f5101161f12f62f
Reusing dataset wikihow (/root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
train_wiki=dataset_wiki['train']
train_wiki

Dataset({
    features: ['text', 'headline', 'title'],
    num_rows: 157252
})

In [19]:
# Visualize the Data

import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(train_wiki)

In [20]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["text"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["headline"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [21]:
sample_size = 20000
data_stats = train_wiki.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (1265 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1057 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2143 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1669 > 1024). Running this sequence through the model will result in indexing errors


In [22]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 750.42735, %-Articles > 512:0.52435, Summary Mean:75.6844, %-Summary > 64:0.43275, %-Summary > 128:0.1454


We can see that on average an article contains 750 tokens with half of the articles being longer than 512. The summary is on average 76 tokens long. 40% our 20000-sample summaries are longer than 64 tokens, and few are longer than 128 tokens.

Bart has limit of `max_length` 1024 on tokenizer, It should be reasonable to use **1024** for tokenized article length and **128** for tokenized summary length for CNN/Daily dataset.